In [ ]:
# !pip install feather-format

In [1]:
import pandas as pd
import numpy as np
import feather
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics

Using TensorFlow backend.


In [2]:
# Write data and labels from Riya's two separate files to a single file
# train_data = feather.read_dataframe("x_train_ADDEDPDMS.feather")
# test_data = feather.read_dataframe("x_test_ADDEDPDMS.feather")
# train_labels = pd.read_csv("y_train_ADDEDPDMS.csv", names=["labels"])
# test_labels = pd.read_csv("y_test_ADDEDPDMS.csv", names=["labels"])
# print(train_data)
# print(test_labels[:10])

# train_data['labels'] = train_labels
# print(train_data)

# test_data["labels"] = test_labels
# print(test_data.shape)

# train_data.to_feather("training_data.feather")
# test_data.to_feather("testing_data.feather")

In [3]:
df = pd.read_csv("../data/exT.csv", nrows=10)
df.head()

,Unnamed: 0,RAB4B,TIGAR,RNF44,DNAH3,RPL23A,ARL8B,CALB2,MFSD3,PIGV,...,SLCO4C1,ARHGAP21,DDX47,POMZP3,SDS,TENM1,CYP4F2,PPP6R1,BATF3,OR8D4
0,Liver.1,2.367912,0.698684,3.931116,0.000000,116.288461,13.839265,0.021383,14.722173,5.437620,...,0.000000,1.199607,0.101155,1.428458,2.471339,0.042357,44.610061,6.394772,0.281878,0
1,Kidney.1,26.299800,2.498500,0.255600,14.944300,2.049600,3.262300,0.146400,14.132900,3.323600,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,Prostate.1,5.592506,2.953563,7.065690,0.012499,325.521932,22.144552,0.356845,28.899361,6.717593,...,0.030674,2.062158,0.185503,8.414909,2.690346,0.688731,0.260365,9.792451,1.671371,0
3,Kidney.2,21.554200,1.334700,1.095300,23.476800,8.741300,11.491700,0.945500,32.975700,6.187900,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,Kidney.3,22.820100,2.640900,3.750800,29.774900,7.595200,15.186900,0.482000,19.233800,8.741800,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [ ]:
df.pop()

In [ ]:
'''
Reading the cell diferentiation data. . .
Each row corresponds to one gene and each column corresponds to reading at a particular time of a day
Transpose is required to map it to the cancer genome data
'''
cell_diff_data = pd.read_csv("../data/exT.csv")
# print(cell_diff.iloc[10:20])
cell_diff_data = cell_diff_data.dropna(axis=1) #remove the NaN columns 
cell_diff_data = cell_diff_data.applymap(lambda x: str(x).strip('gene-')) #each gene name has a prefix 'gene-' so remove that
cell_diff_data = cell_diff_data.T #transpose the dataframe
#Some processing to fix the indexing and column names after transpose
cell_diff_data = cell_diff_data.reset_index(drop=True)
cols = cell_diff_data.iloc[0]
cell_diff_data = cell_diff_data[1:]
cell_diff_data.columns = cols
cell_diff_data = cell_diff_data.reset_index(drop=True)
cell_diff_data.head()

In [ ]:
'''There is something wrong with the way these files are extracted, use tcga_tissue.csv instead'''
# training_data = pd.read_feather("training_data.feather")
# testing_data = pd.read_feather("testing_data.feather")
# # Create a dictionary such that each class of cancer corresponds to an integer
# label_encoder = {name:key for key, name in enumerate(set(training_data["labels"]))}
# # print(label_encoder)
# #Create a decoder to get the class name based on integer value later
# label_decoder = {v:k for k, v in label_encoder.items()}

# train_x = training_data[list(training_data.columns[:-1])]
# print(type(train_x))
# #get the integer label and convert to one hot encoding labels
# train_y = np.array([label_encoder[i] for i in training_data["labels"]])
# train_y = keras.utils.to_categorical(train_y)    
# # print(train_y[:2])



In [ ]:
# # Split the training data into training and validation set 
# #(cannot be done while training because data is sorted by class name in this case)
# x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, shuffle= True)
# print(type(x_train))

# x_test = testing_data[list(training_data.columns[:-1])]
# y_test = np.array([label_encoder[i] for i in testing_data["labels"]])
# y_test = keras.utils.to_categorical(y_test)
# # print(y_test[:2])
# print(x_train.shape)
# print(y_train.shape)
# print(type(x_train))
# len(training_data['labels'].unique())
# x_train[0:10]

In [ ]:
training_data = pd.read_feather("training_data.feather")
training_data.head()

In [ ]:
print(training_data['labels'].unique())

In [ ]:
training_data = pd.read_feather("tcga_tissue.feather")

# Create a dictionary such that each class of cancer corresponds to an integer
label_encoder = {name:key for key, name in enumerate(set(training_data["tissue"]))}
# print(label_encoder)

#Create a decoder to get the class name based on integer value later
label_decoder = {v:k for k, v in label_encoder.items()}

train_x = training_data[list(training_data.columns[1:])]
#get the integer label and convert to one hot encoding labels
train_y = np.array([label_encoder[i] for i in training_data["tissue"]])
print(len(set(train_y)))
train_y = keras.utils.to_categorical(train_y)    

overlapping_cols = np.intersect1d(train_x.columns, cell_diff_data.columns)
print(overlapping_cols.shape)

train_x = train_x[overlapping_cols]

# Split the training data into training and validation set 
#(cannot be done while training because data is sorted by class name in this case)
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, shuffle= True)

# print(y_test[:2])
print(x_train.shape)
print(y_train.shape)
print(type(x_train))
print(training_data['tissue'].unique())
x_train[0:10]

In [ ]:
model = Sequential()
model.add(Dense(1024, activation="relu", input_dim = x_train.shape[1]))
# model.add(Dropout(0.25))
model.add(Dense(512, activation="relu"))
# model.add(Dropout(0.25))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_train.shape[1], activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), verbose=1, epochs=20, shuffle=True)

In [ ]:
valid_loss, valid_accuracy = model.evaluate(x_valid, y_valid)
print(valid_loss, valid_accuracy)

predicted_valid_labels = np.argmax(model.predict(x_valid), axis=1)
valid_labels = np.argmax(y_valid, axis=1)

print("Predicted labels: ", predicted_valid_labels[:10])
print("True labels: ", valid_labels[:10])
print("Predicted labels: ", [label_decoder[i] for i in predicted_valid_labels[:10]])
print("True labels: ", [label_decoder[i] for i in valid_labels[:10]])
# print([(label_decoder[i], label_decoder[j]) for i, j in zip(predicted_valid_labels[:10], valid_labels[:10])])

In [ ]:
pred = model.predict(cell_diff_data[overlapping_cols])
print(pred)
predicted_cell_diff_labels = np.argmax(model.predict(cell_diff_data[overlapping_cols]), axis=1)
print(predicted_cell_diff_labels)
print([label_decoder[i] for i in predicted_cell_diff_labels])

In [ ]:
# Visualization of Confusion Matrix
import seaborn as sns

cm = metrics.confusion_matrix(valid_labels, predicted_valid_labels)
# print(cm)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(20,20))
sns.heatmap(cm_normalized, annot=True, fmt=".4f", linewidths=.5, square = True, cmap = 'summer')
plt.xlabel('Predicted Values', size=20)
plt.ylabel('Actual Values', size=20)

ticks = np.arange(len(set(valid_labels)))
tick_marks = [label_decoder[i] for i in ticks]

plt.xticks(ticks+0.5 ,tick_marks, rotation=90, size=12) #add 0.5 to ticks to position it at center
plt.yticks(ticks+0.5 ,tick_marks, rotation=0, size=12)
all_sample_title = 'Accuracy Score: {:.4f}'.format(valid_accuracy) 
plt.title(all_sample_title, size = 30)



In [ ]:
print(metrics.classification_report(valid_labels, predicted_valid_labels))

In [ ]:
import numpy as np
from sklearn import linear_model, preprocessing

scaler = preprocessing.StandardScaler().fit(x_train)
X = scaler.transform(x_train)
y = np.argmax(y_train, axis=1)
val_X = scaler.transform(x_valid)
val_y = np.argmax(y_valid, axis=1)

lin_sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
lin_sgd.fit(X, y)
lin_sgd.score(val_X, val_y)

In [ ]:
lin_val_pred = lin_sgd.predict(val_X)
print("Predicted valid labels: ", lin_val_pred[10:20])
print("True valid labels: ", val_y[10:20])
print("Predicted valid labels: ", [label_decoder[i] for i in lin_val_pred[10:20]])
print("True valid labels: ", [label_decoder[i] for i in val_y[10:20]])

lin_pred = lin_sgd.predict(cell_diff_data[overlapping_cols])
print(lin_pred)
print("Predicted labels: ", [label_decoder[i] for i in lin_pred])

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0, tol=1e-5)
svc.fit(X, y)
svc.score(val_X, val_y)


In [ ]:
svc_val_pred = svc.predict(val_X)
print("Predicted valid labels: ", svc_val_pred[10:20])
print("True valid labels: ", val_y[10:20])
print("Predicted valid labels: ", [label_decoder[i] for i in svc_val_pred[10:20]])
print("True valid labels: ", [label_decoder[i] for i in val_y[10:20]])

svc_pred = svc.predict(cell_diff_data[overlapping_cols])
print(svc_pred)
print("Predicted labels: ", [label_decoder[i] for i in svc_pred])